In [49]:
# Requirement.
# 1. Connect to sql database using python.
import psycopg2
import pandas as pd
from IPython.display import display

hostname = 'localhost'
username = 'postgres'
password = 'password'
database = 'cards_ingest'
myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )

In [111]:
# 2. Create dataframe by selecting all the cards_ingest.tran_fact
# 
def do_query( conn, query : str ) :
    """
    execute postgresql query on connected database

    :param conn: psycopg2 connection
    :param query: postgresql query string 
    :return: pandas DataFrame for query result
    :rtype: pd.DataFrame
    """
    cur = conn.cursor()
    cur.execute( query )

    # pandas frame
    data = cur.fetchall()
    column_names = [i[0] for i in cur.description]
    frame = pd.DataFrame(data, columns=column_names)
    return frame


query = "SELECT * FROM cards_ingest.tran_fact;"
tran_fact = do_query( myConnection, query )
tran_fact

,tran_id,cust_id,stat_cd,tran_ammt,tran_date
0,102020,cust_101,NY,125.00,2022-01-01
1,102021,cust_101,NY,5125.00,2022-01-01
2,1020321,cust_101,NY,225.00,2022-02-01
3,1020121,cust_101,NY,4125.00,2022-02-03
4,1020222,cust_102,CA,6125.00,2022-01-01
5,1020223,cust_103,CA,7145.00,2022-01-01
6,1023023,cust_103,CA,7145.00,2022-04-01
7,1020123,cust_103,CA,7145.00,2022-03-01
8,1020223,cust_103,CA,7145.00,2022-03-02
9,102024,cust_104,TX,1023.00,2022-01-01


In [131]:
# 3. if the any state has NUll , replace it with TX
tran_fact.loc[(tran_fact['stat_cd']).isnull(), 'stat_cd'] = 'TX'
tran_fact


,tran_id,cust_id,stat_cd,tran_ammt,tran_date,commison
0,102020,cust_101,NY,125.00,2022-01-01,50.0
1,102021,cust_101,NY,5125.00,2022-01-01,2050.0
2,1020321,cust_101,NY,225.00,2022-02-01,90.0
3,1020121,cust_101,NY,4125.00,2022-02-03,1650.0
4,1020222,cust_102,CA,6125.00,2022-01-01,2450.0
5,1020223,cust_103,CA,7145.00,2022-01-01,2858.0
6,1023023,cust_103,CA,7145.00,2022-04-01,2858.0
7,1020123,cust_103,CA,7145.00,2022-03-01,2858.0
8,1020223,cust_103,CA,7145.00,2022-03-02,2858.0
9,102024,cust_104,TX,1023.00,2022-01-01,409.2


In [130]:
# 4. Create another col as commison = tran_ammt*.4
tran_fact['commison'] = tran_fact['tran_ammt'].astype(float) * .4
tran_fact

,tran_id,cust_id,stat_cd,tran_ammt,tran_date,commison
0,102020,cust_101,NY,125.00,2022-01-01,50.0
1,102021,cust_101,NY,5125.00,2022-01-01,2050.0
2,1020321,cust_101,NY,225.00,2022-02-01,90.0
3,1020121,cust_101,NY,4125.00,2022-02-03,1650.0
4,1020222,cust_102,CA,6125.00,2022-01-01,2450.0
5,1020223,cust_103,CA,7145.00,2022-01-01,2858.0
6,1023023,cust_103,CA,7145.00,2022-04-01,2858.0
7,1020123,cust_103,CA,7145.00,2022-03-01,2858.0
8,1020223,cust_103,CA,7145.00,2022-03-02,2858.0
9,102024,cust_104,TX,1023.00,2022-01-01,409.2


In [ ]:
# 5. Create another table as cards_ingest.tran_fact with one extra col


In [ ]:
# 6. Checkk if any null value still exists after the change.


In [ ]:
# 7. Load Data to the new table.


In [ ]:
# 8. Compare the record count from two tables to make sure that same number of records have been loaded.

In [ ]:
# close connection
myConnection.close()

In [ ]:
'''Sample Code to connect:
#!/usr/bin/python

from __future__ import print_function

hostname = 'localhost'
username = 'username'
password = 'password'
database = 'dbname'

# Simple routine to run a query on a database and print the results:
def doQuery( conn ) :
    cur = conn.cursor()

    cur.execute( "SELECT fname, lname FROM employee" )

    for firstname, lastname in cur.fetchall() :
        print( firstname, lastname )


print( "Using psycopg2:" )
import psycopg2
myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
doQuery( myConnection )
myConnection.close()

print( "Using PyGreSQL (pgdb):" )
import pgdb
myConnection = pgdb.connect( host=hostname, user=username, password=password, database=database )
doQuery( myConnection )
myConnection.close()
'''